In [2]:
# ! pip install --user google.cloud
# ! pip install --user google.cloud.speech
# pip install --upgrade google-api-python-client

In [1]:
import os
import io
import pickle
import time

import pandas as pd
import numpy as np

from pydub import AudioSegment
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
from multiprocessing.dummy import Pool
import live2text

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/allanshomefolder/.ssh/EmergencyLocator-4d510abe556a.json"

In [3]:
df_live = live2text.main()

1600

Listening, say "Quit" or "Exit" to stop.

End (ms)       Transcript Results/Status

0: NEW REQUEST


10000: NEW REQUEST
41890: check check still going

20000: NEW REQUEST
48413: check check the still going
50763:  exit

30000: NEW REQUEST
29295: exit
Exiting...


In [5]:
type(df_live)

NoneType

In [5]:
# Instantiates a client
client = speech.SpeechClient()

In [6]:
with open("../data/Pickles/streets.pkl", "rb") as fp:
    streets = pickle.load(fp)

Pool is a class in the multiprocessing package that distributes functionality across multiple processes in a computer. Simply put, it lets the computer assign more than one person to build a fence instead of 1. This dramatically speeds up the time it takes for computationally expensive tasks to run and it called and placed around such tasks. 

In [7]:
pool = Pool(12) 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/allanshomefolder/.ssh/EmergencyLocator-4d510abe556a.json"
client = speech.SpeechClient()

## J's Sandbox

### Google's Live Audio Speech To Text

In [6]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

# Instantiates a client
client = speech.SpeechClient()

# The name of the audio file to transcribe
# file_name = os.path.join(
#     os.path.dirname("https://tunein.com/radio/San-Francisco-49ers-s275697/"),
#     'resources',
#     'audio.raw')
import urllib

testfile = urllib.URLopener()
testfile.retrieve("https://tunein.com/radio/San-Francisco-49ers-s275697/", "/tmp/index.txt")
filename = "https://tunein.com/radio/San-Francisco-49ers-s275697/"

# Loads the audio into memory
with io.open(file_name) as audio_file:
    content = audio_file.read()
    audio = types.RecognitionAudio(content=content)

config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='en-US')

# Detects speech in the audio file
response = client.recognize(config, "https://tunein.com/radio/San-Francisco-49ers-s275697/")

for result in response.results:
    print('Transcript: {}'.format(result.alternatives[0].transcript))

NameError: name 'file_name' is not defined

In [7]:
import os
import speech_recognition as sr
from tqdm import tqdm

with open("/Users/allanshomefolder/keys/EmergencyLocator-4d510abe556a.json") as f:
    GOOGLE_CLOUD_SPEECH_CREDENTIALS = f.read()

r = sr.Recognizer()
files = sorted(os.listdir('parts/'))

all_text = []

for f in tqdm(files):
    name = "parts/" + f
    # Load audio file
    with sr.AudioFile(name) as source:
        audio = r.record(source)
    # Transcribe audio file
    text = r.recognize_google_cloud(audio, credentials_json=GOOGLE_CLOUD_SPEECH_CREDENTIALS)
    all_text.append(text)

transcript = ""
for i, t in enumerate(all_text):
    total_seconds = i * 30
    # Cool shortcut from:
    # https://stackoverflow.com/questions/775049/python-time-seconds-to-hms
    # to get hours, minutes and seconds
    m, s = divmod(total_seconds, 60)
    h, m = divmod(m, 60)

    # Format time as h:m:s - 30 seconds of text
    transcript = transcript + "{:0>2d}:{:0>2d}:{:0>2d} {}\n".format(h, m, s, t)

print(transcript)

with open("transcript.txt", "w") as f:
    f.write(transcript)

FileNotFoundError: [Errno 2] No such file or directory: 'parts/'

In [2]:
import asyncio
import websockets
import json
import threading
from six.moves import queue
from google.cloud import speech
from google.cloud.speech import types


In [2]:
import asyncio
import websockets
import json
import threading
from six.moves import queue
from google.cloud import speech
from google.cloud.speech import types


IP = '0.0.0.0'
PORT = 8000

class Transcoder(object):
    """
    Converts audio chunks to text
    """
    def __init__(self, encoding, rate, language):
        self.buff = queue.Queue()
        self.encoding = encoding
        self.language = language
        self.rate = rate
        self.closed = True
        self.transcript = None

    def start(self):
        """Start up streaming speech call"""
        threading.Thread(target=self.process).start()

    def response_loop(self, responses):
        """
        Pick up the final result of Speech to text conversion
        """
        for response in responses:
            if not response.results:
                continue
            result = response.results[0]
            if not result.alternatives:
                continue
            transcript = result.alternatives[0].transcript
            if result.is_final:
                self.transcript = transcript

    def process(self):
        """
        Audio stream recognition and result parsing
        """
        #You can add speech contexts for better recognition
        cap_speech_context = types.SpeechContext(phrases=["Add your phrases here"])
        client = speech.SpeechClient()
        config = types.RecognitionConfig(
            encoding=self.encoding,
            sample_rate_hertz=self.rate,
            language_code=self.language,
            speech_contexts=[cap_speech_context,],
            model='command_and_search'
        )
        streaming_config = types.StreamingRecognitionConfig(
            config=config,
            interim_results=False,
            single_utterance=False)
        audio_generator = self.stream_generator()
        requests = (types.StreamingRecognizeRequest(audio_content=content)
                    for content in audio_generator)

        responses = client.streaming_recognize(streaming_config, requests)
        try:
            self.response_loop(responses)
        except:
            self.start()

    def stream_generator(self):
        while not self.closed:
            chunk = self.buff.get()
            if chunk is None:
                return
            data = [chunk]
            while True:
                try:
                    chunk = self.buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break
            yield b''.join(data)

    def write(self, data):
        """
        Writes data to the buffer
        """
        self.buff.put(data)


async def audio_processor(websocket, path):
    """
    Collects audio from the stream, writes it to buffer and return the output of Google speech to text
    """
    config = await websocket.recv()
    if not isinstance(config, str):
        print("ERROR, no config")
        return
    config = json.loads(config)
    transcoder = Transcoder(
        encoding=config["format"],
        rate=config["rate"],
        language=config["language"]
    )
    transcoder.start()
    while True:
        try:
            data = await websocket.recv()
        except websockets.ConnectionClosed:
            print("Connection closed")
            break
        transcoder.write(data)
        transcoder.closed = False
        if transcoder.transcript:
            print(transcoder.transcript)
            await websocket.send(transcoder.transcript)
            transcoder.transcript = None

start_server = websockets.serve(audio_processor, IP, PORT)
asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()

RuntimeError: This event loop is already running

In [4]:
asyncio.get_event_loop().stop()

In [ ]:
# Rather than opening an audio file to create the stream (as on line 34 of that example), 
# pass the stream directly to the audio sample object (as on line 36).

## ATL Code

In [54]:
path = '../audio_data/audio_files/wav_files/'
wav_file = []
for filename in os.listdir(path)[1:]:
    wav_file.append(path + filename)
wav_file[:3]

['../audio_data/audio_files/wav_files/10904-20190730-1657_24.wav',
 '../audio_data/audio_files/wav_files/10904-20190730-1657_31.wav',
 '../audio_data/audio_files/wav_files/10904-20190730-1657_23.wav']

Google’s Speech to Text API accepts a wav file and stores it into memory. When calling the speech to text API, we can pass in a dictionary for it to reference its results on. We decided to run the API with and without street context to see if which setting would provide better results. The API then proceeds to detect speech in the audio file and return a transcription of what it heard and it’s confidence in the results. We returned those results as a dataframe. 

## With Street Name Context



In [59]:
def speech_to_text_context(file_name):
    transcript = ''
    conf = 0
    # Loads the audio into memory
    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    #speech_to_text
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code='en-US',
        model="video",

        speech_contexts = [{
                        "phrases": np.random.choice(streets, 5000)
                         }]
    )
    
    # Streaming config
    streaming_config = types.StreamingRecognitionConfig(
        config=config,
        single_utterance=True,
        interim_results =True)
    
    #Detects speech in audio file
    response = client.recognize(config, audio)

    for result in response.results:
        transcript = result.alternatives[0].transcript
        confidence = result.alternatives[0].confidence

    time.sleep(1)
    return transcript, confidence, file_name

In [67]:
pool = Pool(12) 

list_of_transcripts_context = pool.map(speech_to_text_context, wav_file)

captions_context = [a[0] for a in list_of_transcripts_context if a[0] != '']
confidence_context = [a[1] for a in list_of_transcripts_context if a[0] != '']

data_context = {'transcripts': captions_context, 
                'confidence': confidence_context}
df_context = pd.DataFrame(data_context)
# df_context.to_csv('../data/Data/transcribed_radio_with_street_context.csv')
print(df_context.head(25))
    
pool.close()
pool.join()

DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:urllib3.connectionpool:Resetting dropped connection: o

In [68]:
df_context.sample(6)

,transcripts,confidence
6,Peridot we close races here,0.826199
0,Market 1130 Highway Street 1 1 3 0 hi,0.646961
2,powers of,0.584350
5,Grant or kind of somebody can pray Sac State f...,0.698791
3,is going to be,0.461068
1,Ester Drive robson's plane set the driver knoc...,0.797990


In [63]:
np.mean(df_context['confidence'])

0.7162156812846661

## Without Street Name Context

In [60]:
def speech_to_text(file_name):
    transcript = ''
    conf = 0
    # Loads the audio into memory
    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    #speech_to_text
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code='en-US',
        model="video"
    )

    # Detects speech in the audio file    
    response = client.recognize(config, audio)

    for result in response.results:
        transcript = result.alternatives[0].transcript
        confidence = result.alternatives[0].confidence

    time.sleep(1)
    
    return transcript, confidence

In [61]:
pool = Pool(12) 

list_of_transcripts = pool.map(speech_to_text, wav_file)
pool.close()
pool.join()

captions = [a[0] for a in list_of_transcripts if a[0] != '']
confidence = [a[1] for a in list_of_transcripts if a[0] != '']
names = [a[2] for a in list_of_transcripts if a[0] != '']

data = {'transcripts': captions, 
        'confidence': confidence}
df = pd.DataFrame(data)
# df.to_csv('../data/Data/transcribed_radio.csv')
df.head(25)

IndexError: tuple index out of range

In [81]:
np.mean(df['confidence'])

0.7311672965685526

>Code adapted from [NY General Assembly DSI radio-to-location repository](https://github.com/mchbmn/radio-to-location)
